In [1]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Baseline

In [5]:
from sklearn.tree import DecisionTreeClassifier

In [6]:
tree = DecisionTreeClassifier()

In [73]:
X = train.drop('Survived', axis=1)
y = train.Survived

In [74]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocessing

In [75]:
train['PassengerId'] = train.PassengerId.astype(str)

In [76]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,891,891,233,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Survived,891,NaN,NaN,NaN,0.383838,0.486592,0,0,0,1,1
Pclass,891,NaN,NaN,NaN,2.30864,0.836071,1,2,3,3,3
Name,891,891,"Lievens, Mr. Rene Aime",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,891,2,male,577,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,714,NaN,NaN,NaN,29.6991,14.5265,0.42,20.125,28,38,80
SibSp,891,NaN,NaN,NaN,0.523008,1.10274,0,0,0,1,8
Parch,891,NaN,NaN,NaN,0.381594,0.806057,0,0,0,0,6
Ticket,891,681,CA. 2343,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,891,NaN,NaN,NaN,32.2042,49.6934,0,7.9104,14.4542,31,512.329


In [77]:
drop_cols = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived']
keep_cols = [c for c in train.columns if c not in drop_cols]

In [78]:
X = train[keep_cols].copy()
y = train.Survived

In [79]:
cat_cols = ['Sex', 'Embarked']

In [80]:
from sklearn.preprocessing import LabelEncoder

In [81]:
for c in cat_cols:
    X[c] = X[c].astype(str)

In [82]:
lencoders = {}
for c in cat_cols:
    lencoders[c] = LabelEncoder()
    X[c] = lencoders[c].fit_transform(X[c])

In [83]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2


In [84]:
mean_age = X.Age.mean()

In [85]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [86]:
X['AgeMissing'] = X.Age.isnull()
X['Age'] = X.Age.fillna(mean_age)

In [87]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeMissing
0,3,1,22.0,1,0,7.2500,2,False
1,1,0,38.0,1,0,71.2833,0,False
2,3,0,26.0,0,0,7.9250,2,False
3,1,0,35.0,1,0,53.1000,2,False
4,3,1,35.0,0,0,8.0500,2,False


In [88]:
tree.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [89]:
y_pred = tree.predict(X)

In [90]:
len(y_pred), len(X)

(891, 891)

In [91]:
from sklearn.metrics import accuracy_score

In [92]:
train_acc = accuracy_score(y, y_pred)
print(f'Train accuracy: {train_acc:.2%}')

Train accuracy: 98.20%


# Test

In [100]:
test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,418,NaN,NaN,NaN,1100.5,120.81,892,996.25,1100.5,1204.75,1309
Pclass,418,NaN,NaN,NaN,2.26555,0.841838,1,1,3,3,3
Name,418,418,"Fleming, Miss. Honora",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,418,2,male,266,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,332,NaN,NaN,NaN,30.2726,14.1812,0.17,21,27,39,76
SibSp,418,NaN,NaN,NaN,0.447368,0.89676,0,0,0,1,8
Parch,418,NaN,NaN,NaN,0.392344,0.981429,0,0,0,0,9
Ticket,418,363,PC 17608,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,417,NaN,NaN,NaN,35.6272,55.9076,0,7.8958,14.4542,31.5,512.329
Cabin,91,76,B57 B59 B63 B66,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
def process_data(X, keep_cols, cat_cols, lencoders,
                 mean_age):
    X = X[keep_cols].copy()
    for c in cat_cols:
        X[c] = X[c].astype(str)
    for c in cat_cols:
        X[c] = lencoders[c].transform(X[c])
    X['AgeMissing'] = X.Age.isnull()
    X['Age'] = X.Age.fillna(mean_age)
    return X

In [94]:
X_test = process_data(test, keep_cols, cat_cols,
                      lencoders, mean_age)

In [102]:
X_test.isnull().sum()

Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked      0
AgeMissing    0
dtype: int64

In [101]:
X_test['Fare'] = X_test.Fare.fillna(X.Fare.mean())

In [103]:
y_test = tree.predict(X_test)

In [114]:
y_prob = tree.predict_proba(X_test)

In [104]:
len(y_test), len(X_test)

(418, 418)

In [105]:
submission = pd.read_csv('gender_submission.csv')

In [107]:
submission.PassengerId.tolist() == test.PassengerId.tolist()

True

In [109]:
submission['Survived'] = y_test

In [110]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1


In [111]:
submission.to_csv('submission.csv', index=False)

## Lasso

In [116]:
from sklearn.linear_model import Lasso

In [117]:
model = Lasso()

In [118]:
model.fit(X, y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [119]:
model.coef_

array([-0.        , -0.        , -0.        , -0.        ,  0.        ,
        0.00211411, -0.        , -0.        ])